In [11]:
import torch
import yaml
import os

# Humandoid MARL
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.torch_utils import save_models, load_models, load_models_empty
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs
from Humanoid_MARL.utils.utils import load_reward_config, load_config

In [12]:
env_name = "point_mass"
config = load_config(env_name)
env = envs.create(config['env_name'], auto_reset=True, **config['env_config'])
env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['train_config']['device'], get_jax_state=True)

model_path = None
observation = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['train_config']['device'])
env.step(action)
agents = load_models_empty(model_path, Agent, device=config['train_config']['device'], network_config=config['network_config'], training_config=config['train_config'], num_agents=1) 

In [13]:
jax_states = []
num_steps = 1000
eval_reward = 0.0
episodes = torch.zeros((), device = config['train_config']['device'])
for i in range(num_steps):
    print(f"{i} / {num_steps}")
    action = torch.ones(env.action_space.shape[0] * env.num_agents).to(config['train_config']['device']) * 0
    action[1] = 100
    if num_steps > 500:
        action[0] = 0
        action[1] = -10
    else:
        action[0] = 10
        action[1] = 0
    jax_state, observation, reward, done, info = env.step(action)
    episodes += torch.sum(done)
    jax_states.append(jax_state)
    eval_reward += reward
    if done:
        observation = env.reset()
print(f"Total Reward | {eval_reward / episodes}")

0 / 1000
1 / 1000
2 / 1000
3 / 1000
4 / 1000
5 / 1000
6 / 1000
7 / 1000
8 / 1000
9 / 1000
10 / 1000
11 / 1000
12 / 1000
13 / 1000
14 / 1000
15 / 1000
16 / 1000
17 / 1000
18 / 1000
19 / 1000
20 / 1000
21 / 1000
22 / 1000
23 / 1000
24 / 1000
25 / 1000
26 / 1000
27 / 1000
28 / 1000
29 / 1000
30 / 1000
31 / 1000
32 / 1000
33 / 1000
34 / 1000
35 / 1000
36 / 1000
37 / 1000
38 / 1000
39 / 1000
40 / 1000
41 / 1000
42 / 1000
43 / 1000
44 / 1000
45 / 1000
46 / 1000
47 / 1000
48 / 1000
49 / 1000
50 / 1000
51 / 1000
52 / 1000
53 / 1000
54 / 1000
55 / 1000
56 / 1000
57 / 1000
58 / 1000
59 / 1000
60 / 1000
61 / 1000
62 / 1000
63 / 1000
64 / 1000
65 / 1000
66 / 1000
67 / 1000
68 / 1000
69 / 1000
70 / 1000
71 / 1000
72 / 1000
73 / 1000
74 / 1000
75 / 1000
76 / 1000
77 / 1000
78 / 1000
79 / 1000
80 / 1000
81 / 1000
82 / 1000
83 / 1000
84 / 1000
85 / 1000
86 / 1000
87 / 1000
88 / 1000
89 / 1000
90 / 1000
91 / 1000
92 / 1000
93 / 1000
94 / 1000
95 / 1000
96 / 1000
97 / 1000
98 / 1000
99 / 1000
100 / 1000

In [14]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 

In [15]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 